In [1]:
import jupyter_aws_sso
import time
import importlib
importlib.reload(jupyter_aws_sso)

jupyter_aws_sso.login("Jupyter-IR-AdministratorAccess", "383086473915")
#jupyter_aws_sso.login("Jupyter-IR-ViewOnly", "251344881676")

Profile found
If the windows doesn't automatically open, click on this https://device.sso.us-east-1.amazonaws.com/?user_code=MKBG-HSLV to activate the session


<IPython.core.display.Javascript object>

# Organization Configuration
This notebook will configure the organization to have the resources needed to run the Jupyter Incident Response runbooks across the organization. This **must be run in the organizational managemebnt account.**.

The Athena tables can be set up in a delagate account so no resources will be deployed into the management account.

## Accounts:
* **Management Account**: The root account that owns the AWS organization. Typically resources are note deployed to the root account, but some configurations must be done in this account. For example, the cloudtrail organizational trail is created in the management account.
* **Logging**: The account that holds the log files. Typically this account contains the buckets for the CloudTrail and other logs, and Athena tables used to query them.
* **CloudFormation StackSet Delegated Account**: An account designated by the management account to own stacksets.

## Parameters
The following cell holds the parameters for the environment

* **athena_database**: The name of the athena database that will be used to query logs in S3.
* **athena_cloudtrail_table**: The name of the athena table that will be used to query CloutTrail logs in S3.
* **flow_logs_bucket**: The bucket that holds the organizational flow logs. Leave empty and one will be created automatically.
* **enable_flow_logs_automatically**: Automatically enable flow logs in all VPCs in the organization. See [blog](https://aws.amazon.com/blogs/mt/how-to-enable-vpc-flow-logs-automatically-using-aws-config-rules/)


In [2]:
# Save Settings
management_account = "383086473915"
cfn_delegate_account = "383086473915"
logging_account="251344881676"

external_cloud_trail = False # Set this to True if the organization cloudtrail is managed externally.
athena_database = "jupyter-incident-response"  # <--- To customize, replace the value with the athena database name
athena_cloudtrail_table = "cloudtrail_logs"  # <--- To customize, replace the value with the athena table name for cloudtrail logs
flow_logs_bucket = ""  # <--- Leave blank and a bucket will be created automatically
flowlog_output_format = "parquet"
cloudtrail_bucket = ""  # <--- Leave blank and a bucket will be created automatically
enable_flow_logs_automatically = True
opt_in_out = "in"
region_list = "us-east-1,us-west-2,us-east-2"


## CloudFormation
This notebook uses cloudformation to deploy the cloudformation stacks and stack sets used to configure the organization. 

The templates have feature toggles to turn on an off the creation of the configuration. If the resources have already been created outside of this notebook, the resource creation will be skipped.
* **logging-account.yaml**: 
  * If necessary. creates the S3 buckets for organization-wide logging
  * Creates Athena tables to query logs
* **account-level-resources.yaml**:
  * Note: Deployed as a stackset from the **Delegated Stack Stack** account to every account under the root.
  * Creates global resources in each member account.
  * Creates a stackset to deployed to each region in the account
    * Configures Flow Log Auto Remediation
    * 
* **managage-account.yaml**: If necessary, configures organizational CloudTrail logs.

# Organizational Onboarding

The notebook will validate and update the organization to configure the logging and configuration of the organization to the other runbooks have access to the data and services needed to execute the runbooks.

This runbook is designed to be able to be run multiple times, so it will not create additional resources if the configurate has already been found.

The following services will be configured.
* **CloudTrail**: An organization trail will be configured to store the CloudTrail logs in a bucket in the management account. The first trail in each account is included free of charge. If an organzation trail already exists, a new one will not be created.
* **VPC Flow Logs**: Any VPCs with the tag 'runbooks-enable-flowlogs' set to 'true' will have VPC flow logs enabled into a central bucket.
* **Athena**: Athena will be configured to query the following service logs:
  * CloudTrail
  * VPC Flow Logs
  * ***Future*** Application Load Balancer
  * ***Future*** Elastic Load Balancer
  * ***Future*** CloudFront
  * ***Future*** Amazon EMR
  * ***Future*** Global Accelerator
  * ***Future*** GuardDuty
  * ***Future*** Network Firewall
  * ***Future*** Network Load Balancer
  * ***Future*** Route 53
  * ***Future*** Amazon SES
  * ***Future*** AWS WAF
  

## Load Settings

The information on how to connect to data in the AWS environment is managed through SSM Parameter Store.

The athena database and tablenames can be provided below. The runbook will manage the Athena tables so they are configured with the columns and partitioning expected by the notebooks.

In [3]:
# Load Settings
import boto3
ssm_client = boto3.client('ssm')

put_response = ssm_client.put_parameter(Name='Jupyter-Athena-Database', Value=athena_database, Type="String", Overwrite=True)
sso_portal_url_response = ssm_client.put_parameter(Name='Jupyter-Athena-Cloudtrail', Value=athena_cloudtrail_table, Type="String", Overwrite=True)
logging_account_response = ssm_client.put_parameter(Name='Jupyter-Athena-LoggingAccount', Value=logging_account, Type="String", Overwrite=True)

ec2_client = boto3.client('ec2')

stack_set_region_list = ""

if opt_in_out == "out":
    region_opt_out = region_list.split(',')
    stack_set_regions = []

    region_response = ec2_client.describe_regions()

    for region in region_response['Regions']:
        if not region['RegionName'] in region_opt_out:
            stack_set_regions.append(region['RegionName'])
    stack_set_region_list =','.join(stack_set_regions)
else:
    stack_set_region_list = region_list


/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/boto3/compat.py:88: PythonDeprecationWarning: Boto3 will no longer support Python 3.6 starting May 30, 2022. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.7 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)


In [5]:
# Configure Logging Account
org_client = boto3.client('organizations')
desc_client = org_client.describe_organization()

org_id = desc_client['Organization']['Id']

profile = f"Jupyter-IR-AdministratorAccess-{logging_account}"

!aws cloudformation deploy --template-file cfn-templates/logging-account.yaml --stack-name organization-logging --parameter-overrides RegionList=$stack_set_region_list FlowLogsFormat=$flowlog_output_format DatabaseName=$athena_database CloudTrailLogsBucket=$cloudtrail_bucket FlowLogsBucket=$flow_logs_bucket OrgId=$org_id CloudTrailTableName=$athena_cloudtrail_table --capabilities CAPABILITY_IAM --profile $profile


Waiting for changeset to be created..
Waiting for stack create/update to complete
Successfully created/updated stack - organization-logging


**Note** Organizational Rules don't support remediation, so the rule will be deployed in each region.


In [7]:
cfn_bucket = !aws cloudformation describe-stacks --stack-name organization-logging  --profile $profile --query "Stacks[0].Outputs[?OutputKey=='CfnBucket'].OutputValue" --output text
cfn_bucket = cfn_bucket[0]

cfn_cloudtrail_bucket = !aws cloudformation describe-stacks --stack-name organization-logging --profile $profile  --query "Stacks[0].Outputs[?OutputKey=='CloudTrailLogsBucket'].OutputValue" --output text
cfn_flowlog_bucket = !aws cloudformation describe-stacks --stack-name organization-logging --profile $profile  --query "Stacks[0].Outputs[?OutputKey=='BucketName'].OutputValue" --output text

cfn_cloudtrail_bucket = cfn_cloudtrail_bucket[0]
cfn_flowlog_bucket = cfn_flowlog_bucket[0]

print(f"cfn_bucket={cfn_bucket}\ncfn_cloudtrail_bucket={cfn_cloudtrail_bucket}\ncfn_flowlog_bucket={cfn_flowlog_bucket}")


cfn_bucket=organization-logging-cfnbucket-11jngwyngudcw
cfn_cloudtrail_bucket=organization-logging-cfncloudtraillogsbucket-l5ofpp3k3bdm
cfn_flowlog_bucket=organization-logging-flowlogbucket-2c7atewedw2g


## Validate CloudTrail
The next section will check the CloudTrail logs and validate an organizational trail is configured.

Then the cloudformation stack will be created in the management account to configure the organization trail if one is needed.

In [21]:
flowlog_output_format

'parquet'

In [22]:


org_trail_info = {}
org_trail_arn = ""
found_org_trail = True

cloudtrail_client = boto3.client('cloudtrail', region_name = "us-east-1")

if external_cloud_trail == True:
    found_org_trail = False

    paginator = cloudtrail_client.get_paginator('list_trails')

    response_iterator = paginator.paginate()
    for page in response_iterator:
        for trail in page["Trails"]:
            trail_response = cloudtrail_client.get_trail(Name=trail['TrailARN'])
            if trail_response['Trail']['IsOrganizationTrail'] and trail_response['Trail']['IsMultiRegionTrail']:
                found_org_trail = True
                org_trail_info = trail_response['Trail']
                org_trail_arn = org_trail_info['TrailARN']

    if not found_org_trail:
        raise ("External cloud trail not found, please configure.")

!aws cloudformation deploy --template-file cfn-templates/managage-account.yaml --stack-name management-organization --parameter-overrides LoggingAccount=$logging_account CloudTrailArn=$org_trail_arn TrailBucket=$cfn_cloudtrail_bucket --capabilities CAPABILITY_IAM



Waiting for changeset to be created..

No changes to deploy. Stack management-organization is up to date


In [9]:
trail_arn = !aws cloudformation describe-stacks --stack-name management-organization --query "Stacks[0].Outputs[?OutputKey=='TrailArn'].OutputValue" --output text

trail_response = cloudtrail_client.get_trail(Name=trail_arn[0])
org_trail_info = trail_response['Trail']

print(f"Organization Trail was found: {org_trail_info['Name']} in region {org_trail_info['HomeRegion']}")

Organization Trail was found: trail-management-organization in region us-east-1


### Create Trail, if needed
If an organization trail was not found, the following section will create one using a cloudformation template.

## Account and Region Configurations

Any configurations that need to deployed to each account will be handled with this stackset.


# Create / Update the Athena Table for CloudTrail
The next section will create a CloudFormation stack that configures the Athena cloudtrail tables.

Is this region opt-in or opt-out? If opt out, list all the regions available and include those not in the opt-out list.
If opt-in, just pass the region list.


In [10]:

org_unit_id = org_client.list_roots()['Roots'][0]['Id']


In [24]:
import random, string

profile = f"Jupyter-IR-AdministratorAccess-{logging_account}"
random_str=''.join(random.choices(string.ascii_uppercase + string.digits, k=8))

!aws s3 sync cfn-templates/ s3://$cfn_bucket/$random_str/ --profile $profile > /dev/null

profile = f"Jupyter-IR-AdministratorAccess-{cfn_delegate_account}"
template_url = f'https://s3.us-east-1.amazonaws.com/{cfn_bucket}/{random_str}'

!aws cloudformation deploy --stack-name jupyter-ir-config --template-file cfn-templates/organization-ir-config.yaml --parameter-overrides OrgUnitId=$org_unit_id OutputFormat=$flowlog_output_format  AutomateRemediationTemplateUrl=$template_url RegionList=$stack_set_region_list --profile $profile --capabilities CAPABILITY_NAMED_IAM




Waiting for changeset to be created..
Waiting for stack create/update to complete
Successfully created/updated stack - jupyter-ir-config


In [ ]:

profile = f"Jupyter-IR-AdministratorAccess-{logging_account}"
!aws s3 rm --recursive s3://$cfn_bucket/$random_str --profile $profile > /dev/null

## Test CloudTrail access with Athena
The following code uses a helper library to query cloudtrail logs and return them as a dataframe

In [39]:
import jupyter_athena
import importlib
importlib.reload(jupyter_athena)

accountid = "383086473915"

sql = f"""
SELECT * 
FROM \"{athena_database}\".\"{athena_cloudtrail_table}\"
WHERE accountId = '{accountid}'
LIMIT 10
"""

display(jupyter_athena.run_query(sql))


/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/boto3/compat.py:88: PythonDeprecationWarning: Boto3 will no longer support Python 3.6 starting May 30, 2022. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.7 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)


profile_name=Jupyter-IR-ViewOnly-251344881676


,eventversion,useridentity,eventtime,eventsource,eventname,awsregion,sourceipaddress,useragent,errorcode,errormessage,...,eventtype,apiversion,readonly,recipientaccountid,serviceeventdetails,sharedeventid,vpcendpointid,accountid,region,timestamp
0,1.08,"{type=AWSService, principalid=null, arn=null, ...",2022-10-18T06:15:56Z,sts.amazonaws.com,AssumeRole,us-east-1,config.amazonaws.com,config.amazonaws.com,None,None,...,AwsApiCall,None,true,383086473915,None,5e61b01c-20b3-4472-8ae4-99b074acbc0b,None,383086473915,us-east-1,2022/10/18
1,1.08,"{type=AWSService, principalid=null, arn=null, ...",2022-10-18T06:15:56Z,sts.amazonaws.com,AssumeRole,us-east-1,config.amazonaws.com,config.amazonaws.com,None,None,...,AwsApiCall,None,true,383086473915,None,88421475-217c-472f-8118-26af963c8a5b,None,383086473915,us-east-1,2022/10/18
2,1.08,"{type=AWSService, principalid=null, arn=null, ...",2022-10-18T06:15:56Z,sts.amazonaws.com,AssumeRole,us-east-1,config.amazonaws.com,config.amazonaws.com,None,None,...,AwsApiCall,None,true,383086473915,None,0f138bc9-8de8-47f5-93ed-d217d4fdea99,None,383086473915,us-east-1,2022/10/18
3,1.08,"{type=AWSService, principalid=null, arn=null, ...",2022-10-18T06:16:01Z,sts.amazonaws.com,AssumeRole,us-east-1,config.amazonaws.com,config.amazonaws.com,None,None,...,AwsApiCall,None,true,383086473915,None,7873472e-9da4-431a-8717-6065dfb7d43a,None,383086473915,us-east-1,2022/10/18
4,1.08,"{type=AssumedRole, principalid=AROAJX5BEZUSH5I...",2022-10-18T06:16:04Z,s3.amazonaws.com,GetBucketLocation,us-east-1,AWS Internal,AWS Internal,None,None,...,AwsApiCall,None,true,383086473915,None,None,vpce-00dc1369,383086473915,us-east-1,2022/10/18
5,1.08,"{type=AssumedRole, principalid=AROAJX5BEZUSH5I...",2022-10-18T06:16:07Z,s3.amazonaws.com,GetBucketLocation,us-east-1,AWS Internal,AWS Internal,None,None,...,AwsApiCall,None,true,383086473915,None,None,vpce-00dc1369,383086473915,us-east-1,2022/10/18
6,1.08,"{type=AssumedRole, principalid=AROAJX5BEZUSH5I...",2022-10-18T06:16:08Z,s3.amazonaws.com,GetBucketLocation,us-east-1,AWS Internal,AWS Internal,None,None,...,AwsApiCall,None,true,383086473915,None,None,vpce-00dc1369,383086473915,us-east-1,2022/10/18
7,1.08,"{type=AssumedRole, principalid=AROAVSMN3WK5RJD...",2022-10-18T06:15:53Z,config.amazonaws.com,PutEvaluations,us-east-1,fms.amazonaws.com,fms.amazonaws.com,None,None,...,AwsApiCall,None,false,383086473915,None,None,None,383086473915,us-east-1,2022/10/18
8,1.08,"{type=AssumedRole, principalid=AROAJX5BEZUSH5I...",2022-10-18T06:16:14Z,s3.amazonaws.com,GetBucketCors,us-east-1,AWS Internal,AWS Internal,NoSuchCORSConfiguration,The CORS configuration does not exist,...,AwsApiCall,None,true,383086473915,None,None,vpce-00dc1369,383086473915,us-east-1,2022/10/18
9,1.08,"{type=AssumedRole, principalid=AROAJX5BEZUSH5I...",2022-10-18T06:16:14Z,s3.amazonaws.com,GetBucketAcl,us-east-1,AWS Internal,AWS Internal,None,None,...,AwsApiCall,None,true,383086473915,None,None,vpce-00dc1369,383086473915,us-east-1,2022/10/18


## Flow Logs
Enable athena to query organizational flow logs.

This needs to be deployed in two steps. 
1. Deploy CFN The central bucket and athena query in the management account
1. The stacksets for all regions/accounts in organization.

In [49]:


display(jupyter_athena.get_vpc_flow_by_account("913149361159"))


/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/boto3/compat.py:88: PythonDeprecationWarning: Boto3 will no longer support Python 3.6 starting May 30, 2022. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.7 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)


profile_name=Jupyter-IR-ViewOnly-251344881676


,interface_id,srcaddr,srcport,dstaddr,dstport,flow_count,packet_count,sum_bytes
0,eni-05ac490193a65bd0f,89.248.163.168,56419,10.0.0.189,3334,1,1,40
1,eni-05ac490193a65bd0f,5.8.18.77,51892,10.0.0.189,14574,1,1,40
2,eni-05ac490193a65bd0f,189.84.67.142,61833,10.0.0.189,445,1,2,104
3,eni-05ac490193a65bd0f,139.144.239.206,61000,10.0.0.189,448,1,1,40
4,eni-05ac490193a65bd0f,5.8.18.85,51853,10.0.0.189,20923,1,1,40
...,...,...,...,...,...,...,...,...
95,eni-05ac490193a65bd0f,162.142.125.191,34307,10.0.0.189,9299,1,1,44
96,eni-05ac490193a65bd0f,139.144.239.177,61000,10.0.0.189,994,1,1,40
97,eni-05ac490193a65bd0f,162.142.125.188,42408,10.0.0.189,9217,1,1,44
98,eni-05ac490193a65bd0f,51.79.134.226,44772,10.0.0.189,5900,1,1,40


In [42]:
# Delete Flow Logs
import boto3

# delete this in the future or move to another notebook

if False: # Don't do this by default, it exists to delete all the flow logs in an org if the deployment needs to be redone.

    accounts = ["913149361159","251344881676", "959309154643", "669399753706" ]



    for account in accounts:
        profile = f"Jupyter-IR-AdministratorAccess-{account}"
        !aws s3 rm --recursive s3://$cfn_flowlog_bucket/AWSLogs/$account --profile Jupyter-IR-AdministratorAccess-251344881676
        for region_name in stack_set_region_list.split(','):
            session = boto3.session.Session(profile_name=profile, region_name=region_name)
            client = session.client('ec2')
            response = client.describe_flow_logs()

            flow_log_ids = []
            for flow_log in response['FlowLogs']:
                flow_log_id = flow_log['FlowLogId']
                flow_log_ids.append(flow_log_id)

            if len(flow_log_ids) > 0:
                delete_response = client.delete_flow_logs(FlowLogIds=flow_log_ids)
                print(f"deleted: {flow_log_ids} in region {region_name} in account {account}")

In [30]:
for account in accounts:
    profile = f"Jupyter-IR-AdministratorAccess-{account}"
    for region_name in stack_set_region_list.split(','):
        session = boto3.session.Session(profile_name=profile, region_name=region_name)
        client = session.client('ec2')
        response = client.describe_flow_logs()

        flow_log_ids = []
        for flow_log in response['FlowLogs']:
            flow_log_id = flow_log['FlowLogId']
            print(f"FlowLogId: {flow_log_id}:{flow_log['DestinationOptions']['FileFormat']} in region {region_name} in account {account}")

FlowLogId: fl-048fe1e109a86f502:parquet in region us-west-2 in account 913149361159
FlowLogId: fl-0294de065f5c9bf5a:parquet in region us-east-2 in account 913149361159
FlowLogId: fl-00e45ca77c93b25e0:parquet in region us-east-1 in account 251344881676
FlowLogId: fl-0b4a0756571041de5:parquet in region us-east-1 in account 251344881676
FlowLogId: fl-06b9d849764044af0:parquet in region us-east-1 in account 251344881676
FlowLogId: fl-011c48c12e81a3646:parquet in region us-east-2 in account 251344881676
FlowLogId: fl-0be37edafad474ac4:parquet in region us-east-1 in account 959309154643
FlowLogId: fl-0171f17e5489c0381:parquet in region us-east-2 in account 959309154643
FlowLogId: fl-00c59d317d5d8563d:parquet in region us-west-2 in account 669399753706
FlowLogId: fl-0157928ddda3314ae:parquet in region us-west-2 in account 669399753706
